# Create geojson file from list of PUDO coords
See /home/test/Documents/PROJECTS/VFH/iPython_Notebooks/dv_pudo_profiles.ipynb for more info.  

ref: https://geoffboeing.com/2015/10/exporting-python-data-geojson/  
Huom: `with open(output_filename, 'wb') as output_file:` should be `'w'` not `'wb'`  

**Resources**  
geoJSON minifier: https://jsonformatter.org/json-minify  

In [1]:
import sys
sys.executable

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker

from IPython.display import HTML, display
import datetime
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os

from datetime import datetime

import json

# Functions

In [3]:
# https://geoffboeing.com/2015/10/exporting-python-data-geojson/
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

# GeoJSON properties for file

In [4]:
# Define the properties of the PUDO geojson file
prop = ['type','counts', 'overlap']

# Variables

In [5]:
ward=10
this_day = "Monday"
this_tod = "nightIII"

In [6]:
if ward == 1:
    test_id1 = 13454935
    test_id2 = 13453946
elif ward == 10:
    test_id1 = 13467106 # Adelaide and John
    test_id2 = 13466303

# Read PUDO data from csv file

In [7]:
fname = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/all_pudo_mon.csv'
# fname = '../../../Documents/PROJECTS/VFH/COT_PAGE/test_data/all_pudo_mon_ampeak.csv'
# fname = '~/Documents/PROJECTS/BDITTO/VFH/DATA_STORY/DATA/from_pgAdmin/all_pudo_mon.csv'

df = pd.read_csv(fname)

print(df.shape)
df.head(5)

(494842, 14)


,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
0,2018-09-03 00:00:00,2018-09-03 00:34:01,18.0,10.0,13447475,13467654,Monday,nightII,0101000020E61000005BE2AF6670DA53C01D22DBEA93E2...,0101000020E610000013E6CE309BD953C0938FE7777DD2...,43.770139,-79.413110,43.644454,-79.400097
1,2018-09-03 00:00:00,2018-09-03 00:10:54,10.0,13.0,13467763,13464030,Monday,nightII,0101000020E610000051E519D30AD953C0C23B89A05BD2...,0101000020E610000079E6DB3E68D853C00F3AF08356D5...,43.643421,-79.391286,43.666703,-79.381363
2,2018-09-03 00:00:00,2018-09-03 00:12:34,13.0,10.0,13464551,13467763,Monday,nightII,0101000020E6100000A342F6F3C6D753C0E8EE4CFEEDD4...,0101000020E610000051E519D30AD953C0C23B89A05BD2...,43.663513,-79.371518,43.643421,-79.391286
3,2018-09-03 00:00:00,2018-09-03 00:24:18,13.0,3.0,13465140,13470547,Monday,nightII,0101000020E6100000DF5023ED39D853C020ED2E3976D4...,0101000020E610000004410BFFE5E153C041B447898CCC...,43.659858,-79.378536,43.598039,-79.529663
4,2018-09-03 00:00:00,2018-09-03 00:18:21,8.0,13.0,14044860,14673305,Monday,nightII,0101000020E6100000A37FC1EFB8DC53C070C432B4FCDB...,0101000020E61000008C84E11C68D853C062D98093A3D5...,43.718649,-79.448788,43.669054,-79.381355


In [8]:
list(df)

['pickup_datetime',
 'dropoff_datetime',
 'pickup_ward2018',
 'dropoff_ward2018',
 'pickup_gc_intersection_id',
 'dropoff_gc_intersection_id',
 'dow',
 'timewindow',
 'pickup_geom',
 'dropoff_geom',
 'lat_pickup',
 'lon_pickup',
 'lat_dropoff',
 'lon_dropoff']

In [9]:
df.loc[(df['pickup_gc_intersection_id']==13454935) & (df['dow']=='Monday') & (df['timewindow']=='amPeak')]

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
333377,2018-09-10 08:00:00,2018-09-10 08:08:53,1.0,1.0,13454935,20232410,Monday,amPeak,0101000020E61000005851FD2F49E653C0139F29DE82DC...,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,43.722744,-79.598217,43.728927,-79.60573
350828,2018-09-10 08:00:00,2018-09-10 08:05:53,1.0,1.0,13454935,13459809,Monday,amPeak,0101000020E61000005851FD2F49E653C0139F29DE82DC...,0101000020E61000000DB97F50AAE553C0D1F8C0B9BFD8...,43.722744,-79.598217,43.693351,-79.58852


In [10]:
df.loc[(df['dropoff_gc_intersection_id']==13454935) & (df['dow']=='Monday') & (df['timewindow']=='amPeak')]

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
428572,2018-09-03 08:00:00,2018-09-03 08:03:28,1.0,1.0,13455196,13454935,Monday,amPeak,0101000020E61000005B2EED6468E753C00EA1543F52DC...,0101000020E61000005851FD2F49E653C0139F29DE82DC...,43.72126,-79.615747,43.722744,-79.598217


# Housekeeping

## 1. Round lat, lon to 4 decimal places

In [11]:
n=4 # number of decimals to round to

df['lat_pickup'] = df['lat_pickup'].round(decimals=n)
df['lon_pickup'] = df['lon_pickup'].round(decimals=n)

df['lat_dropoff'] = df['lat_dropoff'].round(decimals=n)
df['lon_dropoff'] = df['lon_dropoff'].round(decimals=n)

## 2. Filter out rows where neither pickup or dropoff wards are the ward of interest
Make col `keep` = 1 for where pick-up or for drop-off wards are the ward of interest

In [12]:
# Assign a Pickup or Dropoff in column 'type'
df['keep'] = np.nan
df['latitude'] = np.nan
df['longitude'] = np.nan

df.loc[(df['pickup_ward2018']==ward) | (df['dropoff_ward2018']==ward), 'keep'] = 1

In [13]:
# Check
df['keep'].unique()

array([ 1., nan])

In [14]:
# Keep only rows where `keep` is not null
df = df[df['keep'].notnull()]

In [15]:
# Check
df['keep'].unique()

array([1.])

In [16]:
print(df.shape)

(166086, 17)


In [17]:
df.head(5)

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff,keep,latitude,longitude
0,2018-09-03 00:00:00,2018-09-03 00:34:01,18.0,10.0,13447475,13467654,Monday,nightII,0101000020E61000005BE2AF6670DA53C01D22DBEA93E2...,0101000020E610000013E6CE309BD953C0938FE7777DD2...,43.7701,-79.4131,43.6445,-79.4001,1.0,NaN,NaN
1,2018-09-03 00:00:00,2018-09-03 00:10:54,10.0,13.0,13467763,13464030,Monday,nightII,0101000020E610000051E519D30AD953C0C23B89A05BD2...,0101000020E610000079E6DB3E68D853C00F3AF08356D5...,43.6434,-79.3913,43.6667,-79.3814,1.0,NaN,NaN
2,2018-09-03 00:00:00,2018-09-03 00:12:34,13.0,10.0,13464551,13467763,Monday,nightII,0101000020E6100000A342F6F3C6D753C0E8EE4CFEEDD4...,0101000020E610000051E519D30AD953C0C23B89A05BD2...,43.6635,-79.3715,43.6434,-79.3913,1.0,NaN,NaN
7,2018-09-03 00:00:00,2018-09-03 00:25:12,10.0,23.0,13467149,13445567,Monday,nightII,0101000020E6100000114B23960FD953C0EB083B20E4D2...,0101000020E610000021F13FBAB7D153C0B3D1137822E4...,43.6476,-79.3916,43.7823,-79.2768,1.0,NaN,NaN
8,2018-09-03 00:00:00,2018-09-03 00:13:19,10.0,10.0,14621058,13467746,Monday,nightII,0101000020E6100000363D49368ED853C0D81E02B61FD2...,0101000020E610000045A22B6CE1DA53C095E3C7F86DD2...,43.6416,-79.3837,43.6440,-79.4200,1.0,NaN,NaN


## 3a. Split df into two dfs for pickups and dropoffs separately
one where pickup ward == ward, the other where dropoff_ward == ward  

Assign `type` = 1 for pickup df, 5 for dropoff df.

In [18]:
df_pu = pd.DataFrame()
df_pu = df.loc[df['pickup_ward2018']==ward]
df_pu['type'] = 1

/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df_do = pd.DataFrame()
df_do = df.loc[df['dropoff_ward2018']==ward]
df_do['type'] = 5

/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
df_pu.head(4)

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff,keep,latitude,longitude,type
1,2018-09-03 00:00:00,2018-09-03 00:10:54,10.0,13.0,13467763,13464030,Monday,nightII,0101000020E610000051E519D30AD953C0C23B89A05BD2...,0101000020E610000079E6DB3E68D853C00F3AF08356D5...,43.6434,-79.3913,43.6667,-79.3814,1.0,NaN,NaN,1
7,2018-09-03 00:00:00,2018-09-03 00:25:12,10.0,23.0,13467149,13445567,Monday,nightII,0101000020E6100000114B23960FD953C0EB083B20E4D2...,0101000020E610000021F13FBAB7D153C0B3D1137822E4...,43.6476,-79.3916,43.7823,-79.2768,1.0,NaN,NaN,1
8,2018-09-03 00:00:00,2018-09-03 00:13:19,10.0,10.0,14621058,13467746,Monday,nightII,0101000020E6100000363D49368ED853C0D81E02B61FD2...,0101000020E610000045A22B6CE1DA53C095E3C7F86DD2...,43.6416,-79.3837,43.6440,-79.4200,1.0,NaN,NaN,1
10,2018-09-03 00:00:00,2018-09-03 00:07:58,10.0,10.0,13467929,13467729,Monday,nightII,0101000020E6100000855DE4035AD853C096EC9BA52BD2...,0101000020E610000035A3F68DF5D853C00B8F00E564D2...,43.6420,-79.3805,43.6437,-79.3900,1.0,NaN,NaN,1


In [21]:
# Fill `latitude` and `longitude` col with pickup or dropoff lat/lon
df_pu['latitude'] = df_pu['lat_pickup']
df_pu['longitude'] = df_pu['lon_pickup']

df_do['latitude'] = df_do['lat_dropoff']
df_do['longitude'] = df_do['lon_dropoff']

/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [22]:
df_pu.head(4)

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff,keep,latitude,longitude,type
1,2018-09-03 00:00:00,2018-09-03 00:10:54,10.0,13.0,13467763,13464030,Monday,nightII,0101000020E610000051E519D30AD953C0C23B89A05BD2...,0101000020E610000079E6DB3E68D853C00F3AF08356D5...,43.6434,-79.3913,43.6667,-79.3814,1.0,43.6434,-79.3913,1
7,2018-09-03 00:00:00,2018-09-03 00:25:12,10.0,23.0,13467149,13445567,Monday,nightII,0101000020E6100000114B23960FD953C0EB083B20E4D2...,0101000020E610000021F13FBAB7D153C0B3D1137822E4...,43.6476,-79.3916,43.7823,-79.2768,1.0,43.6476,-79.3916,1
8,2018-09-03 00:00:00,2018-09-03 00:13:19,10.0,10.0,14621058,13467746,Monday,nightII,0101000020E6100000363D49368ED853C0D81E02B61FD2...,0101000020E610000045A22B6CE1DA53C095E3C7F86DD2...,43.6416,-79.3837,43.6440,-79.4200,1.0,43.6416,-79.3837,1
10,2018-09-03 00:00:00,2018-09-03 00:07:58,10.0,10.0,13467929,13467729,Monday,nightII,0101000020E6100000855DE4035AD853C096EC9BA52BD2...,0101000020E610000035A3F68DF5D853C00B8F00E564D2...,43.6420,-79.3805,43.6437,-79.3900,1.0,43.6420,-79.3805,1


In [23]:
df_do.head(4)

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff,keep,latitude,longitude,type
0,2018-09-03 00:00:00,2018-09-03 00:34:01,18.0,10.0,13447475,13467654,Monday,nightII,0101000020E61000005BE2AF6670DA53C01D22DBEA93E2...,0101000020E610000013E6CE309BD953C0938FE7777DD2...,43.7701,-79.4131,43.6445,-79.4001,1.0,43.6445,-79.4001,5
2,2018-09-03 00:00:00,2018-09-03 00:12:34,13.0,10.0,13464551,13467763,Monday,nightII,0101000020E6100000A342F6F3C6D753C0E8EE4CFEEDD4...,0101000020E610000051E519D30AD953C0C23B89A05BD2...,43.6635,-79.3715,43.6434,-79.3913,1.0,43.6434,-79.3913,5
8,2018-09-03 00:00:00,2018-09-03 00:13:19,10.0,10.0,14621058,13467746,Monday,nightII,0101000020E6100000363D49368ED853C0D81E02B61FD2...,0101000020E610000045A22B6CE1DA53C095E3C7F86DD2...,43.6416,-79.3837,43.6440,-79.4200,1.0,43.6440,-79.4200,5
10,2018-09-03 00:00:00,2018-09-03 00:07:58,10.0,10.0,13467929,13467729,Monday,nightII,0101000020E6100000855DE4035AD853C096EC9BA52BD2...,0101000020E610000035A3F68DF5D853C00B8F00E564D2...,43.6420,-79.3805,43.6437,-79.3900,1.0,43.6437,-79.3900,5


## 3b. Aggregate pickups and dropoffs

In [24]:
df_pu_agg=pd.DataFrame()

df_pu_agg = df_pu.groupby(
    ['type','dow','timewindow','latitude','longitude','pickup_gc_intersection_id']
    ).size().reset_index(name='counts')

df_pu_agg.head(3)

,type,dow,timewindow,latitude,longitude,pickup_gc_intersection_id,counts
0,1,Monday,amPeak,43.6311,-79.4155,30021269,1
1,1,Monday,amPeak,43.6316,-79.4273,13469141,1
2,1,Monday,amPeak,43.6320,-79.4195,20229234,1


In [25]:
# Check
df_pu_agg.loc[df_pu_agg['pickup_gc_intersection_id']==13454935]

,type,dow,timewindow,latitude,longitude,pickup_gc_intersection_id,counts


In [26]:
df_do_agg=pd.DataFrame()

df_do_agg = df_do.groupby(
    ['type','dow','timewindow','latitude','longitude','dropoff_gc_intersection_id']
    ).size().reset_index(name='counts')

df_do_agg.head(3)

,type,dow,timewindow,latitude,longitude,dropoff_gc_intersection_id,counts
0,5,Monday,amPeak,43.6309,-79.4198,13469187,1
1,5,Monday,amPeak,43.6311,-79.4155,30021269,2
2,5,Monday,amPeak,43.6312,-79.4102,13969590,2


In [27]:
# Check
df_do_agg.loc[df_do_agg['dropoff_gc_intersection_id']==13454935]

,type,dow,timewindow,latitude,longitude,dropoff_gc_intersection_id,counts


## 4. Make a common table on common columns

### a. Aggregates
Concat `df_pu_agg` and `df_do_agg`.

In [28]:
# First rename intersection id to common name to allow concat
df_pu_agg = df_pu_agg.rename(columns={'pickup_gc_intersection_id': 'intersection_id'})
df_do_agg = df_do_agg.rename(columns={'dropoff_gc_intersection_id': 'intersection_id'})

In [29]:
cols = ['type','dow','timewindow','latitude','longitude','intersection_id','counts']

df_pudo_agg=pd.DataFrame()

df_pudo_agg=pd.concat([df_pu_agg[cols], df_do_agg[cols] ])

df_pudo_agg.head(4)

,type,dow,timewindow,latitude,longitude,intersection_id,counts
0,1,Monday,amPeak,43.6311,-79.4155,30021269,1
1,1,Monday,amPeak,43.6316,-79.4273,13469141,1
2,1,Monday,amPeak,43.6320,-79.4195,20229234,1
3,1,Monday,amPeak,43.6322,-79.4005,13469040,2


In [30]:
# Check
df_pudo_agg.loc[(df_pudo_agg['timewindow']=='amPeak') & (df_pudo_agg['intersection_id']==test_id1)]

,type,dow,timewindow,latitude,longitude,intersection_id,counts
415,1,Monday,amPeak,43.6479,-79.3903,13467106,62
404,5,Monday,amPeak,43.6479,-79.3903,13467106,13


In [31]:
df_pudo_agg.loc[(df_pudo_agg['timewindow']=='amPeak') & (df_pudo_agg['intersection_id']==test_id2)]

,type,dow,timewindow,latitude,longitude,intersection_id,counts
589,1,Monday,amPeak,43.6528,-79.3934,13466303,3


## 4. Subset this_tod

In [32]:
df_pudo_agg_tod = df_pudo_agg.loc[(df_pudo_agg['timewindow']==this_tod)]

In [33]:
# Mark the >= second occurrence of overlapping intersections for this_tod as True
df_pudo_agg_tod['first overlap'] = df_pudo_agg_tod['intersection_id'].duplicated()

/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
# Check
df_pudo_agg_tod.loc[(df_pudo_agg_tod['intersection_id']==test_id1)]

,type,dow,timewindow,latitude,longitude,intersection_id,counts,first overlap
2874,1,Monday,nightIII,43.6479,-79.3903,13467106,21,False
2797,5,Monday,nightIII,43.6479,-79.3903,13467106,14,True


In [35]:
# Check
df_pudo_agg_tod.loc[(df_pudo_agg_tod['intersection_id']==test_id2)]

,type,dow,timewindow,latitude,longitude,intersection_id,counts,first overlap
2918,5,Monday,nightIII,43.6528,-79.3934,13466303,1,False


In [36]:
# Store overlapping intersections in a list
overlap_ids = list(df_pudo_agg_tod.loc[df_pudo_agg_tod['first overlap']==True, 'intersection_id'])
print(len(overlap_ids))

372


In [37]:
# Check
test_id2 in list(overlap_ids)

# => False is correct

False

In [38]:
# Mark all occurrences of overlapping intersections as True by finding if `intersection_id` is in
# list `overlap_ids`
df_pudo_agg_tod['overlap'] = df_pudo_agg_tod['intersection_id'].isin(list(overlap_ids))

/home/test/anaconda3/envs/vfh/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
# Check
df_pudo_agg_tod.loc[(df_pudo_agg_tod['intersection_id']==test_id1)]

,type,dow,timewindow,latitude,longitude,intersection_id,counts,first overlap,overlap
2874,1,Monday,nightIII,43.6479,-79.3903,13467106,21,False,True
2797,5,Monday,nightIII,43.6479,-79.3903,13467106,14,True,True


In [40]:
df_pudo_agg_tod.loc[(df_pudo_agg_tod['intersection_id']==test_id2)]

,type,dow,timewindow,latitude,longitude,intersection_id,counts,first overlap,overlap
2918,5,Monday,nightIII,43.6528,-79.3934,13466303,1,False,False


In [41]:
# => Checks are OK

In [42]:
# df=

# Save to geojson

## Aggregated points

In [43]:
geojson_pu_unique = df_to_geojson(df_pudo_agg_tod.loc[(df_pudo_agg_tod['type']==1) & 
                                                      (df_pudo_agg_tod['overlap']==False)], ['counts'])

In [44]:
geojson_pudo = df_to_geojson(df_pudo_agg_tod.loc[(df_pudo_agg_tod['type']==1) & 
                                                      (df_pudo_agg_tod['overlap']==True)], ['counts'])

In [45]:
geojson_do_unique = df_to_geojson(df_pudo_agg_tod.loc[(df_pudo_agg_tod['type']==5) & 
                                                      (df_pudo_agg_tod['overlap']==False)], ['counts'])

# Write geojson to file

## Aggregated points

In [46]:
print('w' +repr(ward), this_day, this_tod)

w10 Monday nightIII


In [47]:
output_filename = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/INDIVIDUAL_GEOJSONS/w' + repr(ward) + '_092018_' + this_day + '_' + this_tod + '_pu_agg_unique.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_pu_unique, output_file, indent=2) 

In [48]:
output_filename = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/INDIVIDUAL_GEOJSONS/w' + repr(ward) + '_092018_' + this_day + '_' + this_tod + '_do_agg_unique.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_do_unique, output_file, indent=2) 

In [49]:
output_filename = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/INDIVIDUAL_GEOJSONS/w' + repr(ward) + '_092018_' + this_day + '_' + this_tod + '_pudo_agg.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_pudo, output_file, indent=2) 

`
$ ls -lh ~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/INDIVIDUAL_GEOJSONS/
total 216K
-rw-r--r-- 1 test test 109K Dec  9 11:23 w1_092018_Monday_pmPeak_do_agg.geojson
-rw-r--r-- 1 test test 104K Dec  9 11:23 w1_092018_Monday_pmPeak_pu_agg.geojson
`


`
$ ls -lh src/geojson/
-rw-r--r-- 1 test test 328K Dec  4 15:46 w1_092018_Monday_amPeak_agg.geojson
-rw-r--r-- 1 test test 753K Dec  4 13:51 w1_092018_Monday_amPeak.geojson
`
